In [1]:
import numpy, matplotlib, time, copy, random
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

In [2]:
class Library():
    def __init__(self,index,N,T,M, book_ids: set):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = book_ids
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
        
        

def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = [int(n) for n in content[1].split()]#tuple([int(n) for n in content[1].split()])
    libraries = []
    def book_sort(book):
        return bookValues[book]
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = tuple(sorted([int(id) for id in content[2+2*i+1].split()], key = book_sort, reverse=True))
        libraries.append(Library(i,N,T,M,book_ids))
    return ((B,L,D), bookValues, libraries)

In [3]:
def check_solution(D, libraries):
    days = 0
    prev_books = set()
    for library in libraries:
        days+=library.signup_time
        if len(library.book_ids) > (D-days)*library.books_per_day:
            print("what", len(library.book_ids), (D-days)*library.books_per_day)
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days < D)

In [12]:
def get_best_solution(file_path):
    (B,L,D),book_values, libraries = process_file(file_path)
    
    def get_score(selected_lib_ids):
        local_book_values = copy.copy(book_values)
        day = 0
        already_scanned_books  = set()
        def library_score(lib):
            delta_time = D - lib.signup_time
            n_best_books = lib.book_ids[:min(len(lib.book_ids), delta_time*lib.books_per_day)]
            sum_of_best_book_scores = sum([local_book_values[book] for book in n_best_books])
            return sum_of_best_book_scores/lib.signup_time
        sorted_libraries = sorted([copy.copy(libraries[i]) for i in selected_lib_ids],key=library_score, reverse=True)
        it=0;
        while it < len(sorted_libraries):
            next_library = sorted_libraries[it]
            cutted_b_ids = []
            for book_id in next_library.book_ids:
                if not( book_id in already_scanned_books):
                    cutted_b_ids.append(book_id)
            next_library.book_ids = tuple(cutted_b_ids)
            day+=next_library.signup_time
            if day>=D:
                break
            next_library.book_ids = next_library.book_ids[:min(len(next_library.book_ids), (D-day)*next_library.books_per_day)]
            already_scanned_books.update(next_library.book_ids)
            it+=1
        score = 0
        for i in range(0,it):
            for idk in sorted_libraries[i].book_ids:
                score+=book_values[idk]    
        check_solution(D, sorted_libraries[:it])
#         print("Scoer ", score)
        return score
    
    max_score = get_score([i for i in range(len(libraries))])
    return max_score
    
sum_scores = 0
for file_path in file_paths:
    print(file_path)
    %timeit -n 3 get_best_solution(file_path)
print("SUM SCORES", sum_scores)

a_example.txt
96.6 µs ± 58.2 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
b_read_on.txt
77.9 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
c_incunabula.txt
183 ms ± 4.19 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
d_tough_choices.txt
383 ms ± 5.88 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
e_so_many_books.txt
289 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
f_libraries_of_the_world.txt
335 ms ± 6.88 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
SUM SCORES 0
